In [ ]:
import time
from tools import *
from models import *
from make_data import load_dict_from_json

In [2]:
database = load_dict_from_json('data/M0A_train_data.json')
test_sample = select_random_keys(database)

In [3]:
themes = {
    "T1": "Conscience",
    "T2": "Desire",
    "T3": "Freedom",
    "T4": "Goodness",
    "T5": "Identity",
    "T6": "Justice",
    "T7": "Language",
    "T8": "Meaning",
    "T9": "Science",
    "T10": "Technology",
    "T11": "Truth",
    "T12": "Time",
    "T13": "Existence",
    "T14": "Music",
    "T15": "Imagination",
    "T16": "The Unconscious",
    "T17": "Education",
    "T18": "Body & Mind",
    "T19": "Beauty",
    "T20": "Art",
    "T21": "Love",
    "T22": "Reality",
    "T23": "Politics",
    "T24": "Work",
    "T25": "Living Together",
    "T26": "Philosophy",
    "T27": "Matter",
    "T28": "Death",
    "T29": "Religion",
    "T30": "History",
    "T31": "Thought",
    "T32": "Madness",
    "T33": "Joy & Happiness"
}

## Test Logic

In [ ]:
def execute_test(prompts, test_sample, database, themes):
    for prompt_name, prompt in prompts.items():
        delay_between_batches = 0  # Time (in seconds) to wait between batches
        system_instructions = prompt

        def process_in_batches(test_sample, delay_between_batches, system_instructions):

            results = {}

            for text in test_sample:
                allocation = get_theme_allocation_gemini(text, system_instructions).replace(']', '').replace('[', '').replace(' ', '').split(',')
                results[text] = [themes[allocation] for allocation in allocation]

                time.sleep(delay_between_batches)
            return results

        results_dict = process_in_batches(test_sample, delay_between_batches, system_instructions)

        save_answers_to_sqlite('data/models_answers.db', f"{prompt_name}", results_dict, database)

        y_pred = create_binary_matrix(themes, [item for key, item in results_dict.items()])
        y_true = create_binary_matrix(themes, [database[key] for key in test_sample])

        jaccard_index, hamming_loss = evaluate_metrics(y_true, y_pred)
        personalized_loss = personalized_metric(y_true, y_pred)[0]

        prompt_metrics_save(prompt_name, prompt, jaccard_index, hamming_loss, personalized_loss)

        return results_dict

## Test!

In [ ]:
prompts = {'test': """

You are a professor of the Humanities (Litterature, Philosophy, Poetry mostly) and as such, you are giving the task to classify texts related to the humanities by themes. You have been provided with texts from which you must exclusively derive its theme(s).

You are given the text of a page from a digital book and a fixed list of 33 themes. Your task is to determine which of these themes are relevant to the content of the page.
A page can be allocated to more than one theme. However, a page CANNOT be allocated to more than six themes. Only allocate a theme if you are very confident that it is a relevant theme.
Remember that it is better to have less relevant themes allocated than many that are irrelevant!
Return the allocation in the format: [T<theme_number>, T<theme_number>, ...]
Below are the 33 themes along with their guiding questions and descriptions:

T1. Consciousness
Is consciousness a uniquely human trait and to what extent our individual conscience is shaped by social influences?

T2. Desire
The relationship between desire, satisfaction, and human nature, examining whether desires can be fulfilled, consciously known, their impact on our wellbeing, and their influence on our behavior and pursuit of truth and goodness.

T3. Freedom
The relationship between consciousness, personal autonomy, and determinism, examining whether awareness and choice are truly liberating factors in human freedom.

T4. Goodness
Aspects of moral goodness, examining whether education, perception, intention, and human nature influence ethical behavior and moral development.

T5. Identity
Facets of personal identity, examining how self-awareness, change, relationships, work, and choices contribute to our understanding of who we are.

T6. Justice
Dimensions of justice, examining its relationship with freedom, law, conventions, experience, state power, moral choices, and democratic systems.

T7. Language
The relationship between language and thought, examining whether language serves as a barrier, tool, or mediator in human communication, understanding, and expression.

T8. Meaning
What has meaning ?

T9. Science
The fundamental questions raisend by science about knowledge, truth, and human understanding - from the possibility of scientific knowledge about life to the relationship between reason, experience, belief, and certainty.

T10. Technology
What is the relationship between technological progress, human freedom, and our connection to nature?

T11. Truth
Dimensions of truth, including its dependence on human perception, methods of verification, relationship with science and politics, and the nature of certainty and doubt.

T12. Time
Aspects of time, including its relationship with freedom, happiness, destruction, human limitations, knowledge, efficiency, novelty, and leisure.

T13. Existence
Existence encompasses both action and contemplation, raising questions about how we engage with life's moments and opportunities.

T14. Music

T15. Imagination
Does imagination enrich knowledge and what it means to lack of imagination?

T16. The Unconscious
The compatibility of the unconscious with freedom, its relationship with self-awareness, and its influence on human expression.

T17. Education
The relationship between culture, human nature, and personal development, examining whether cultural education liberates us, shapes our character, influences our happiness, and affects our moral development.

T18. Body & Mind
What difference can be made between the mind and the body?

T19. Beauty
Dimensions of beauty, from its transformative power on consciousness to its relationship with utility, happiness, and religious experience.

T20. Art
Aspects of art, including its relationship with understanding, truth, reality, education, language, beauty, meaning, joy and necessity.

T21. Love
Dimensions of love, including its rationality, universality, self-knowledge, and distinctions from other forms of human connection.

T22. Reality
The nature of reality and our ability to perceive and understand it, examining aspects like perception, the reliability of appearances, intuition, judgment, and the distinction between dreams and reality.

T23. Politics
What are the foundations and limits of political authority and human social organization?

T24. Work
Questions about work, examining its necessity, social impact, virtue, time value, and technical nature.

T25. Living Together
Questions exploring various aspects of social living, including moral obligations, duty, conflict, responsibility, and the relationship between individual and collective happiness.

T26. Philosophy
The relationship between philosophy and fundamental concepts like happiness, governance, and religion.

T27. Matter
Does the mind have access to matter and what is matter?

T28. Death
What are the fundamental questions about mortality and our ability to comprehend and accept death?

T29. Religion
Aspects of religion's necessity for humanity, its relationship with reason, and its cultural origins.

T30. History
Questions about history's nature, examining its scientific status, its relevance to the future, its role in political decision-making, and the agency behind historical events.

T31. Thought
Thought: its limitations, the nature of ideas, and our ability to comprehend origins.

T32. Madness

T33. Joy & Happiness
Aspects of happiness, including its relationship with truth, consciousness, culture, and well-being, examining whether happiness is achievable, personal, or compatible with the realities of existence.

DO NOT provide explanations, only provide the allocation code line.
DO NOT provide the title of each theme such as [T7. Language, T5. Identity], only provide the theme code such as [T7, T5].
Only allocate a theme if you are very confident that it is relevant.

           """}

In [ ]:
# res_dict = execute_test(prompts, test_sample, database, themes)